In [1]:
#libraries

import os
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Bidirectional
from keras.layers.embeddings import Embedding
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
import re
re.compile('<title>(.*)</title>')
from nltk.stem.snowball import SnowballStemmer
from sklearn.utils import shuffle



Using TensorFlow backend.


In [2]:
#loading the news items

charlie = pd.read_csv('charlie.csv')
charlie = shuffle(charlie)
ferguson = pd.read_csv('ferguson.csv')
ferguson=shuffle(ferguson)
german = pd.read_csv('germanwings.csv')
german=shuffle(german)
ottawashooting = pd.read_csv('ottawa.csv')
ottawashooting=shuffle(ottawashooting)
sydneysiege = pd.read_csv('sydneysiege.csv')
sydneysiege=shuffle(sydneysiege)



In [3]:
#exctracting labels

labels = charlie['class'].map(lambda x : 1 if int(x) > 0 else 0)
labels1 = ferguson['class'].map(lambda x : 1 if int(x) > 0 else 0)
labels2 = german['class'].map(lambda x : 1 if int(x) > 0 else 0)
labels3 = ottawashooting['class'].map(lambda x : 1 if int(x) > 0 else 0)
labels4 = sydneysiege['class'].map(lambda x : 1 if int(x) > 0 else 0)




In [4]:
#cleaning text by removing faaltu cheeze

def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text





In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vaibhav.singh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
#calling the cleaning fx

charlie['text'] = charlie['text'].map(lambda x: clean_text(x))
ferguson['text'] = ferguson['text'].map(lambda x: clean_text(x))
german['text'] = german['text'].map(lambda x: clean_text(x))
ottawashooting['text'] = ottawashooting['text'].map(lambda x: clean_text(x))
sydneysiege['text'] = sydneysiege['text'].map(lambda x: clean_text(x))



In [6]:

#appending the documents as whole

documents = charlie['text']
#print(documents.shape)
documents = documents.append(ferguson['text'],ignore_index = True)
#print(documents.shape)
documents = documents.append(german['text'],ignore_index = True)
#print(documents.shape)
documents = documents.append(ottawashooting['text'],ignore_index = True)
#print(documents.shape)
documents = documents.append(sydneysiege['text'],ignore_index = True)
#print(documents.shape)



In [7]:

#appending labels.
labels=labels.append(labels1,ignore_index=True)
labels=labels.append(labels2,ignore_index=True)
labels=labels.append(labels3,ignore_index=True)
labels=labels.append(labels4,ignore_index=True)



In [8]:
docs = charlie['text']
lab = charlie['class'].map(lambda x : 1 if int(x) > 0 else 0)





In [9]:




# originally only for charlie hebdo  documents = charlie['text']

vocabulary_size = 50000
tokenizer = Tokenizer()
tokenizer.fit_on_texts(docs)
vocab_size = len(tokenizer.word_index)+1

sequences = tokenizer.texts_to_sequences(docs)
data = pad_sequences(sequences, maxlen=70)

print(data.size)

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=70))
model.add(Bidirectional(LSTM(50, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
model.add(Bidirectional(LSTM(200, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
model.add(Bidirectional(LSTM(400, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())













145460
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 70, 100)           512600    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 70, 100)           60400     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 70, 200)           160800    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 70, 400)           641600    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 70, 800)           2563200   
_________________________________________________________________
flatten_1 (Fla

In [10]:

model.fit(np.array(data),np.array(lab), validation_split=0.09, epochs=5)



Instructions for updating:
Use tf.cast instead.
Train on 1890 samples, validate on 188 samples
Epoch 1/5
1890/1890 [==============================] - 51s 27ms/step - loss: 0.5796 - acc: 0.7751 - val_loss: 0.3481 - val_acc: 0.8457
Epoch 2/5
1890/1890 [==============================] - 45s 24ms/step - loss: 0.2525 - acc: 0.8868 - val_loss: 0.1959 - val_acc: 0.9362
Epoch 3/5
1890/1890 [==============================] - 45s 24ms/step - loss: 0.0956 - acc: 0.9661 - val_loss: 0.5104 - val_acc: 0.8617
Epoch 4/5
1890/1890 [==============================] - 45s 24ms/step - loss: 0.0773 - acc: 0.9788 - val_loss: 0.4452 - val_acc: 0.8936
Epoch 5/5
1890/1890 [==============================] - 45s 24ms/step - loss: 0.0230 - acc: 0.9947 - val_loss: 0.3806 - val_acc: 0.8883
